In [1]:
#Testing boto3 aws connection

import boto3

s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

crownfloats-test.com
elasticbeanstalk-us-east-1-862738521567
elasticbeanstalk-us-west-2-862738521567
hidden-6ix.com
loginfo-hidden6ix.com
redditpostestimator
www.hidden-6ix.com


In [2]:
###Starting with exercise 2, to provide observability and feedback on performance of task 1 

In [73]:
import datetime
import time
ec2_client = boto3.client('ec2') 
cloudwatch_client = boto3.client('cloudwatch')



In [36]:
#2a) See the status of all the diffferent servers being monitored
instance_status_paginator = ec2_client.get_paginator('describe_instance_status')
instance_status_iterator = instance_status_paginator.paginate()
instances = []
for instance_status in instance_status_iterator:
    for status in instance_status['InstanceStatuses']:
        instances.append(status)
        print(status['InstanceId'])
        print(status['InstanceState'])

i-0c6474917143e0522
{'Code': 16, 'Name': 'running'}


In [33]:
#2a) See how many processes are running on it
# Methodology is to have hosts run process that continually emits cloud watch metrics, which can be watched and alerted on


In [80]:
##############
# Script which emits cloudwatch metrics 
##############
while(True): 
    
    response = cloudwatch_client.put_metric_data(
        Namespace='chorus',
        MetricData=[
        {
            'MetricName': 'process1',  #Add more metrics for important proceses to observe
            'Dimensions': [
                {
                    'Name': 'isImportProcessRunning',
                    'Value': '1'
                },
            ],
            'StatisticValues': {
                   'SampleCount': 1,
                'Sum': 1,
                'Minimum': 1,
                'Maximum': 1
            }
        },
        ]
    )
    print('submitted')
    time.sleep(1)
    



submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted
submitted


KeyboardInterrupt: 

In [82]:
# Polling instance level cloudwatch metrics (it can take up to 15 minutes to see results)
response = cloudwatch_client.list_metrics(
    Namespace="chorus"
)
print(response)
for metric in response['Metrics']:
    print(metric)
    response = cloudwatch_client.get_metric_data(
        MetricDataQueries=[
            {
            'Id': 'chorusAITechnical',
            'MetricStat': {
                'Metric': metric,
                'Period': 300,
                'Stat': 'Sum',
                'Unit': 'None'
               },
            },
        ],
    StartTime=datetime.datetime(2015, 1, 1),
    EndTime=datetime.datetime(2019, 12, 12))
    print(response)
    print("======================================================")


{'Metrics': [{'Namespace': 'chorus', 'MetricName': 'importProcessRunning', 'Dimensions': [{'Name': 'isImportProcessRunning', 'Value': '1'}]}, {'Namespace': 'chorus', 'MetricName': 'process1', 'Dimensions': [{'Name': 'isImportProcessRunning', 'Value': '1'}]}], 'ResponseMetadata': {'RequestId': '8f50c9ef-72a4-11e9-a1db-0d43e0b61dc2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '8f50c9ef-72a4-11e9-a1db-0d43e0b61dc2', 'content-type': 'text/xml', 'content-length': '834', 'date': 'Thu, 09 May 2019 21:51:10 GMT'}, 'RetryAttempts': 0}}
{'Namespace': 'chorus', 'MetricName': 'importProcessRunning', 'Dimensions': [{'Name': 'isImportProcessRunning', 'Value': '1'}]}
{'MetricDataResults': [{'Id': 'chorusAITechnical', 'Label': 'importProcessRunning', 'Timestamps': [datetime.datetime(2019, 5, 9, 21, 40, tzinfo=tzutc()), datetime.datetime(2019, 5, 9, 21, 30, tzinfo=tzutc())], 'Values': [1.0, 200.0], 'StatusCode': 'PartialData'}], 'NextToken': 'ANUVeafcc1fwvCPLh4YhyJETEqmgu9sOdknDP2yucWQ+

In [84]:
#### 1 a) Monitor spot prices

response = ec2_client.describe_spot_price_history(
)
print(response['SpotPriceHistory'])

[{'AvailabilityZone': 'us-west-2c', 'InstanceType': 'p2.xlarge', 'ProductDescription': 'SUSE Linux', 'SpotPrice': '0.380400', 'Timestamp': datetime.datetime(2019, 5, 9, 21, 49, 19, tzinfo=tzutc())}, {'AvailabilityZone': 'us-west-2c', 'InstanceType': 'p2.xlarge', 'ProductDescription': 'Linux/UNIX', 'SpotPrice': '0.280400', 'Timestamp': datetime.datetime(2019, 5, 9, 21, 49, 19, tzinfo=tzutc())}, {'AvailabilityZone': 'us-west-2b', 'InstanceType': 'i3en.24xlarge', 'ProductDescription': 'SUSE Linux', 'SpotPrice': '4.250300', 'Timestamp': datetime.datetime(2019, 5, 9, 21, 49, 3, tzinfo=tzutc())}, {'AvailabilityZone': 'us-west-2b', 'InstanceType': 'i3en.24xlarge', 'ProductDescription': 'Linux/UNIX', 'SpotPrice': '4.150300', 'Timestamp': datetime.datetime(2019, 5, 9, 21, 49, 3, tzinfo=tzutc())}, {'AvailabilityZone': 'us-west-2a', 'InstanceType': 'm4.16xlarge', 'ProductDescription': 'SUSE Linux', 'SpotPrice': '1.139100', 'Timestamp': datetime.datetime(2019, 5, 9, 21, 48, 50, tzinfo=tzutc())}, {